In [1]:
import sys,time
# sys.path.insert(0,'/kaggle/input/akh-sglang')
import os
# os.environ['PYTHONPATH'] = '{}:/kaggle/input/akh-sglang'.format(os.environ["PYTHONPATH"])
# os.environ["TRITON_PTXAS_PATH"] = "/kaggle/working/ptxas"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import shutil
import openai
import math, numpy as np

start_time = time.time()
cutoff_time = time.time() + (4 * 60 + 56) * 60
cutoff_times = [int(x) for x in np.linspace(cutoff_time, start_time+60*60, 50 + 1)]
import gc
# import polars as pl
import warnings
import pandas as pd
import re
import collections
import torch
# import kaggle_evaluation.aimo_2_inference_server
pd.set_option('display.max_colwidth', None)
from datasets import load_dataset
warnings.simplefilter('ignore')

In [2]:
import concurrent.futures
import sys
import io
import multiprocessing

In [3]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

In [4]:
# # Path to the original file
# src_path = "/kaggle/input/akh-sglang/triton/backends/nvidia/bin/ptxas"
# # Path to the writable directory
# dst_path = "/kaggle/working/ptxas"

# # Copy the file
# shutil.copy(src_path, dst_path)
# os.chmod(dst_path, 0o755)

In [5]:
server_process = execute_shell_command(
    "python3 -m sglang.launch_server --model-path akh99/r1-2_1 --port 30020 --host 0.0.0.0  --mem-fraction-static 0.8"
)

wait_for_server("http://localhost:30020")

2025-03-29 18:18:06.008396: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 18:18:06.019979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743272286.032651   34057 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743272286.036530   34057 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743272286.047817   34057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [6]:
def run_code_with_timeout(code, timeout=10):
    def execute_code(queue, code):
        """
        Function to execute code and store the output or error in a queue.
        """
        try:
            # Redirect stdout to capture the output
            old_stdout = sys.stdout
            sys.stdout = io.StringIO()

            try:
                # Execute the provided code
                exec(code)
                output = sys.stdout.getvalue()
                queue.put({ "output": output, "error": "" })
            except Exception as e:
                queue.put({ "output": "", "error": str(e) })

            # Restore stdout
            sys.stdout = old_stdout
        except Exception as e:
            queue.put({ "output": "", "error": str(e) })
    
    # Create a queue to capture the result from the subprocess
    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=execute_code, args=(queue, code))

    # Start the process
    process.start()
    # Wait for the process to finish or timeout
    process.join(timeout)

    if process.is_alive():
        # If the process is still alive after the timeout, terminate it
        process.terminate()
        process.join()
        return { "output": "", "error": "Time limit of 10sec exceeded!" }
    if not queue.empty():
        return queue.get()
    else:
        return { "output": "", "error": "Unknown error" }

In [7]:
client = openai.Client(base_url="http://127.0.0.1:30020/v1", api_key="None")
# llm_model_pth = "/kaggle/input/deepseek-r1/transformers/deepseek-r1-distill-qwen-7b-awq-casperhansen/1"
llm_model_pth = "akh99/r1-2_1"
max_model_len = 8192 * 3 // 2 
BS = 10
rep = 16
min_p = 0.0
top_p = 0.95
temperature = 0.6

In [8]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")

def get_length(text):
    return len(tok(text, add_special_tokens = False)["input_ids"])

In [9]:
def extract_boxed_texts(text):
    text = text.strip("\n")
    pattern = r'\\boxed{((?:[^{}]|{[^{}]*})+)}'
    matches = re.findall(pattern, text)
    if not matches:
        return -1000
    ans = []
    for content in matches:
        if content.isdigit():
            num = int(content)
            ans.append(num % 1000)

    if len(ans) > 0: return ans[-1]
    return -1000

In [10]:
def run_infer(message, max_model_len = max_model_len, temperature=temperature):
    
    response = client.chat.completions.create(
        model=llm_model_pth,
        messages=message,
        temperature=temperature,
        max_tokens=max_model_len,
        stop=[ "</code>", "</think>"],
        top_p=top_p,
    )

    # return [s.message.content for s in responses.choices] 
    return response.choices[0].message.content

In [11]:
def get_tokens_after_last_code(text):
    parts = re.split(r"<code>", text)
    return parts[-1] if len(parts) > 1 else []

def add_output_at_end(response, tool_calls = 0):
    code = get_tokens_after_last_code(response)
    output_json = run_code_with_timeout(code)
    print("code runned")
    if len(output_json["error"]) == 0:
        return response + "</code>\n<output>\n{}\n</output>\n".format(output_json["output"].strip("\n"))
    else:   
        return response + "</code>\n<output>\nnError: {}\n</output>\n".format(output_json["error"].strip("\n")) 
        # + """Wait, I need to make sure the code is appropriate. Let me correct the code based on the error.```python\n"""

def run_code_with_timeout(code, timeout=10):
    def execute_code(queue, code):
        """
        Function to execute code and store the output or error in a queue.
        """
        try:
            # Redirect stdout to capture the output
            old_stdout = sys.stdout
            sys.stdout = io.StringIO()

            try:
                # Execute the provided code
                exec(code)
                output = sys.stdout.getvalue()
                queue.put({ "output": output, "error": "" })
            except Exception as e:
                queue.put({ "output": "", "error": str(e) })

            # Restore stdout
            sys.stdout = old_stdout
        except Exception as e:
            queue.put({ "output": "", "error": str(e) })
    
    # Create a queue to capture the result from the subprocess
    queue = multiprocessing.Queue()
    process = multiprocessing.Process(target=execute_code, args=(queue, code))

    # Start the process
    process.start()
    # Wait for the process to finish or timeout
    process.join(timeout)

    if process.is_alive():
        # If the process is still alive after the timeout, terminate it
        process.terminate()
        process.join()
        return { "output": "", "error": "Time limit of 10sec exceeded!" }
    if not queue.empty():
        return queue.get()
    else:
        return { "output": "", "error": "Unknown error" }

In [12]:
def check_code(response):
    code_open_generated_freq = response.count("<code>")
    code_close_generated_freq = response.count("</code>")
    if code_open_generated_freq == code_close_generated_freq:
        return 0
    elif code_open_generated_freq == code_close_generated_freq+1:
        return 1
    else:
        return 2
        
def solve_one(problem, max_model_len = max_model_len, temperature=temperature):
    initial_message = [
            {"role": "system", "content": "Please think step-by-step and give final answer using \\boxed{}, after taking modulo 1000."},
            {"role": "user", "content": problem}
        ] 
    TOTAL_TOKENS_UNTIL_NOW = 0
    MAX_TOKENS = max_model_len
    MAX_TURNS = 3
    TURNS = 0
    CURRENT_RESPONSE = ""
    code_status = -1
    
    while(1):
        if TURNS >= MAX_TURNS or MAX_TOKENS - TOTAL_TOKENS_UNTIL_NOW<=0:
            return CURRENT_RESPONSE
        if TURNS == 3:
            print("able to do 3 turns")
        if code_status == -1:
            response = run_infer(initial_message + [{"role":"assistant", "content": CURRENT_RESPONSE}],  max_model_len = MAX_TOKENS - TOTAL_TOKENS_UNTIL_NOW)
            CURRENT_RESPONSE = CURRENT_RESPONSE + response  
            TOTAL_TOKENS_UNTIL_NOW += get_length(response)
            code_status = check_code(response)
        elif code_status == 0:
            return CURRENT_RESPONSE
        elif code_status == 1:
            CURRENT_RESPONSE = add_output_at_end(CURRENT_RESPONSE)
            response = run_infer(initial_message +[{"role":"assistant", "content":CURRENT_RESPONSE}], max_model_len = MAX_TOKENS - TOTAL_TOKENS_UNTIL_NOW)
            CURRENT_RESPONSE = CURRENT_RESPONSE + response  
            TOTAL_TOKENS_UNTIL_NOW += get_length(response)
            TURNS += 1
            code_status = check_code(response)
        else:
            print("code tokens didn't match..")
            # response = run_infer(list_of_messages + [{"role":"assistant", "content": CURRENT_RESPONSE}])
            return CURRENT_RESPONSE

In [13]:
def parallelize(fn, *args):
    """
    Parallelizes the execution of a function over multiple lists of arguments.
    
    Parameters:
        fn (callable): The function to be executed.
        *args (list): One or more lists of arguments. The i-th elements from each list
                      are passed together to fn.
                      
    Returns:
        list: A list of results from applying fn on each set of arguments.
    """
    if not args:
        return []
    
    # Ensure all lists have the same length
    n = len(args[0])
    if any(len(lst) != n for lst in args):
        raise ValueError("All input lists must have the same length.")

    results = [None] * n
    with concurrent.futures.ThreadPoolExecutor(max_workers=200) as executor:
        # Submit each job with the corresponding elements from all lists.
        future_to_index = {
            executor.submit(fn, *[arg[i] for arg in args]): i
            for i in range(n)
        }
        # Retrieve results as they complete.
        for future in concurrent.futures.as_completed(future_to_index):
            index = future_to_index[future]
            try:
                results[index] = future.result()
            except Exception as e:
                results[index] = str(e)
                print(str(e))
    return results

In [14]:
def is_list_of_strings(var):
    return isinstance(var, list) and all(isinstance(item, str) for item in var)
    
def predict_for_question(question, rep = rep):
    
    # if time.time() > cutoff_time:
    #     return 210
   
    # else:
    # solns = solve(question, rep = 24, max_model_len = 8192 * 3 // 2)
    
    is_list = is_list_of_strings(question)
    if not is_list:
        problems = [question]*rep
        solns = parallelize(solve_one, problems)
        # ans = do_maj_vote(solns)
        # cutoff_times.pop()
        return solns
    else:
        problems = []
        for q in question:
            for idx in range(rep):
                problems.append(q)

        solns = parallelize(solve_one, problems)
        # ans = do_maj_vote(solns)
        # cutoff_times.pop()
        return solns

# elif time.time() > cutoff_times[-1]:
    #     # print("Speedrun!")
    #     solns = solve(question, rep = 10, max_model_len = 8192)
    #     ans = do_maj_vote(solns)
    #     cutoff_times.pop()
    #     return ans

In [15]:
def break_ties(keys):
    lesser_than_100 = []
    greater_than_100 = []
    for key in keys:
        if key < 100:
            lesser_than_100.append(key)
        else:
            greater_than_100.append(key)

    if len(greater_than_100) == 0:
        return keys[0]
    elif len(lesser_than_100) == 0:
        return keys[0]
    else:
        return greater_than_100[0]

In [16]:
def do_maj_vote(outputs):
    output = [extract_boxed_texts(soln) for soln in outputs]
    print("answers",output)
    d = collections.defaultdict(int)
    for item in output:
        if item != -1000:
            d[item] += 1
    
    max_key = -10000
    max_value = -10000
    
    if len(d)==0:
        return 210
    
    for key in d:
        if(d[key]>max_value):
            max_key = key
            max_value = d[key]
    
    tie_keys = []
    for key in d:
        if(d[key]==max_value):
            tie_keys.append(key)

    if len(tie_keys) == 1:
        return max_key
    elif len(tie_keys) > 1:
        return break_ties(tie_keys)
    else:
        return max_key    
        

In [17]:
# a = predict_for_question(["What is 3+7?", "What is 3 + 7**2?"])

In [18]:
# [extract_boxed_texts(text) for text in a]

In [19]:
import pandas as pd
df = pd.read_csv("reference.csv")

In [20]:
from tqdm import tqdm
start = 0
end = len(df)

outputs = []
answers = []
lengths = []

for i in tqdm(range(10)):
    # try:
        solns = predict_for_question( list(df["problem"])[i] )
        outputs.extend(solns)
        answers.extend([extract_boxed_texts(soln) for soln in solns])
        # lengths.extend([len(tokenizer(soln)["input_ids"]) for soln in solns])
        print(answers)
    # except:
    #     print("exception occured at: ", i)

  0%|          | 0/10 [00:00<?, ?it/s]

[2025-03-29 18:19:45 TP0] Prefill batch. #new-seq: 1, #new-token: 117, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-29 18:19:45 TP0] Prefill batch. #new-seq: 13, #new-token: 13, #cached-token: 1521, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-03-29 18:19:45 TP0] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 234, token usage: 0.00, #running-req: 14, #queue-req: 0, 
[2025-03-29 18:19:45 TP0] Decode batch. #running-req: 16, #token: 631, token usage: 0.00, gen throughput (token/s): 61.19, #queue-req: 0, 
[2025-03-29 18:19:45 TP0] Decode batch. #running-req: 16, #token: 1271, token usage: 0.00, gen throughput (token/s): 2878.68, #queue-req: 0, 
[2025-03-29 18:19:45 TP0] Decode batch. #running-req: 16, #token: 1911, token usage: 0.00, gen throughput (token/s): 2869.56, #queue-req: 0, 
[2025-03-29 18:19:46 TP0] Decode batch. #running-req: 16, #token: 2551, token usage: 0.00, gen throughput (token/s): 2865.24, #queue-req: 0, 
[2025-03

 10%|█         | 1/10 [01:25<12:48, 85.38s/it]

[-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000]
[2025-03-29 18:21:10 TP0] Prefill batch. #new-seq: 1, #new-token: 90, #cached-token: 24, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-03-29 18:21:10 TP0] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 226, token usage: 0.00, #running-req: 1, #queue-req: 0, 
[2025-03-29 18:21:10 TP0] Prefill batch. #new-seq: 4, #new-token: 4, #cached-token: 452, token usage: 0.00, #running-req: 3, #queue-req: 0, 
[2025-03-29 18:21:10 TP0] Prefill batch. #new-seq: 9, #new-token: 9, #cached-token: 1017, token usage: 0.00, #running-req: 7, #queue-req: 0, 
[2025-03-29 18:21:10 TP0] Decode batch. #running-req: 16, #token: 470, token usage: 0.00, gen throughput (token/s): 1182.98, #queue-req: 0, 
[2025-03-29 18:21:11 TP0] Decode batch. #running-req: 16, #token: 1110, token usage: 0.00, gen throughput (token/s): 2874.45, #queue-req: 0, 
[2025-03-29 18:21:11 TP0] Decode batch.

 20%|██        | 2/10 [02:49<11:15, 84.47s/it]

...

 90%|█████████ | 9/10 [12:05<01:19, 79.50s/it]

[2025-03-29 18:31:51 TP0] Decode batch. #running-req: 1, #token: 12426, token usage: 0.01, gen throughput (token/s): 162.64, #queue-req: 0, 
[2025-03-29 18:31:51] INFO:     127.0.0.1:37202 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, 0, 250, -1000, 250, -1000, 750, -1000, -1000, -1000, -1000, -1000, -1000, 250, 0, -1000, -1000, 180, 180, 180, 180, 180, -1000, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 143, 143, -1000, 143, 143, 143, -1000, 143, 143, 143, 143, 143, -1000, 777, 143, 143, -1000, -1000, 1, -1000, -1000, -1000, -1000, 1, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, 891, 892, 891, 901, 910, 861, -1000, 893, 891, 18, 900, 882, 891, -1000, 909, 80, -1000, -1000, 0, -1000, -1000, -1000, 0, -1000, -1000, -1000, -1000, -1000, -1000, -1000, 

100%|██████████| 10/10 [13:22<00:00, 78.64s/it]

[2025-03-29 18:33:07 TP0] Decode batch. #running-req: 4, #token: 49164, token usage: 0.03, gen throughput (token/s): 636.79, #queue-req: 0, 
[2025-03-29 18:33:07] INFO:     127.0.0.1:46718 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-29 18:33:07] INFO:     127.0.0.1:46744 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-29 18:33:07] INFO:     127.0.0.1:46768 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-03-29 18:33:07] INFO:     127.0.0.1:46826 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, 0, 250, -1000, 250, -1000, 750, -1000, -1000, -1000, -1000, -1000, -1000, 250, 0, -1000, -1000, 180, 180, 180, 180, 180, -1000, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 143, 143, -1000, 143, 143, 143, -1000, 143, 143, 143, 143, 143, -1000, 777, 143, 143, -1000, -1000, 1, -1000, -1000, -1000, -1000, 1, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1

100%|██████████| 10/10 [13:22<00:00, 80.27s/it]


In [21]:
for i in range(10):
    start = 16*i
    end = 16*(i+1)
    print(do_maj_vote(outputs[start:end]))

answers [-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000]
210
answers [0, 250, -1000, 250, -1000, 750, -1000, -1000, -1000, -1000, -1000, -1000, 250, 0, -1000, -1000]
250
answers [180, 180, 180, 180, 180, -1000, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180]
180
answers [143, 143, -1000, 143, 143, 143, -1000, 143, 143, 143, 143, 143, -1000, 777, 143, 143]
143
answers [-1000, -1000, 1, -1000, -1000, -1000, -1000, 1, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000]
1
answers [-1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000]
210
answers [891, 892, 891, 901, 910, 861, -1000, 893, 891, 18, 900, 882, 891, -1000, 909, 80]
891
answers [-1000, -1000, 0, -1000, -1000, -1000, 0, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000, -1000]
0
answers [201, 4, 201, 201, 201, 202, 201, 201, 200, 200, -1000, 201, 201, 201, 202, 201]
201
answers [902, 902, 902, -1000

In [25]:
count

0

In [24]:
# for i in range(10):
count = 0
for output in outputs[:]:
        if "<code>" in output:
            print(output[-250:])
            count += 1
        # print(output[-1000:])
            print("----------------------------")

In [ ]:
# from tqdm import tqdm
# start = 0
# end = len(df)

# outputs = []
# answers = []
# lengths = []

# for i in tqdm(range(start, end, BS)):
#     try:
#         solns = solve(list(df[question_column][i:i+BS]))
#         outputs.extend(solns)
#         answers.extend([extract_boxed_texts(soln) for soln in solns])
#         lengths.extend([len(tokenizer(soln)["input_ids"]) for soln in solns])
#         print(answers)
#     except:
#         print("exception occured at: ", i)

# ################ Store results ################
# duration = time.time() - start_time

# outputs_batched = []
# answers_batched = []
# lengths_batched = []

# N = len(outputs)
# for i in range(0, N, reps):
#     outputs_batch = []
#     answers_batch = []
#     lengths_batch = []
#     for j in range(reps):
#         outputs_batch.append(outputs[i+j])
#         answers_batch.append(answers[i+j])
#         lengths_batch.append(lengths[i+j])
        
#     outputs_batched.append(outputs_batch)
#     answers_batched.append(answers_batch)
#     lengths_batched.append(lengths_batch)

# df["outputs"] = outputs_batched
# df["gen_answers"] = answers_batched
# df["output_lengths"] = lengths_batched
# df.to_parquet(f"{doc_name}_full_preds.parquet", index=None)

# targets = list(df[answer_column])

# eval_obj = evaluate_all(answers_batched, lengths_batched, targets)
# eval_obj.print_all_metrics()

# print(f"duration = {duration}")
# print(f"Average tokens consumed = {sum(lengths)/len(lengths)}")

In [ ]:
# # Replace this function with your inference code.
# # The function should return a single integer between 0 and 999, inclusive.
# # Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
# def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
#     id_ = id_.item(0)
#     print("------")
#     question = question.item(0)
#     answer = predict_for_question(question)
#     print(question, answer)
#     print("------\n\n\n")
#     return pl.DataFrame({'id': id_, 'answer': answer})

In [ ]:
# %%time
# for qn in list(df["problem"]):
#     answer = predict_for_question(qn)
#     print(answer)

In [ ]:
# ONLY runnig time for 10 ref prblms:
                
# VLLM = 40 mins
# VLLM v1 = 31 mins
# SGLang = 28mins

In [ ]:
# pd.read_csv(
#     '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
# ).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [ ]:
# inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)
# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     inference_server.serve()
# else:
#     inference_server.run_local_gateway(
#         (
#             'reference.csv',
#         )
#     )